**Install Dependencies:**

In [1]:
%%capture --no-stderr
%pip install -U --quiet pyautogen sentence_transformers pypdf chromadb langchain langchain-community langchain_chroma markdownify

**LLM Calling:**

In [ ]:
import time

from typing_extensions import Annotated

import autogen
from autogen.cache import Cache
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
config_list = [{"model":"llama3-8b-8192"
,#"llama-3.1-8b-instant",#"llama-3.1-70b-versatile",#
                "api_key":os.environ.get("GROQ_API_KEY"),"base_url": "https://api.groq.com/openai/v1/"}]
llm_config={"config_list" : config_list}

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


**Agent Example**

In [ ]:
llm_config = {
    "config_list": config_list,
}
# from autogen import
coder = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. You have a stopwatch and a timer, these tools can and should be used in parallel. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="A proxy for the user for executing code.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)

# define functions according to the function description

# An example async function registered using register_for_llm and register_for_execution decorators


@user_proxy.register_for_execution()
@coder.register_for_llm(description="create a timer for N seconds")
async def timer(num_seconds: Annotated[str, "Number of seconds in the timer."]) -> str:
    for i in range(int(num_seconds)):
        time.sleep(1)
        # should print to stdout
    return "Timer is done!"


# An example sync function registered using register_function
def stopwatch(num_seconds: Annotated[str, "Number of seconds in the stopwatch."]) -> str:
    for i in range(int(num_seconds)):
        time.sleep(1)
    return "Stopwatch is done!"


autogen.agentchat.register_function(
    stopwatch,
    caller=coder,
    executor=user_proxy,
    description="create a stopwatch for N seconds",
)

**Agent's tools**

In [ ]:
coder.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'create a timer for N seconds',
   'name': 'timer',
   'parameters': {'type': 'object',
    'properties': {'num_seconds': {'type': 'string',
      'description': 'Number of seconds in the timer.'}},
    'required': ['num_seconds']}}},
 {'type': 'function',
  'function': {'description': 'create a stopwatch for N seconds',
   'name': 'stopwatch',
   'parameters': {'type': 'object',
    'properties': {'num_seconds': {'type': 'string',
      'description': 'Number of seconds in the stopwatch.'}},
    'required': ['num_seconds']}}}]

**Run:**

In [ ]:

with Cache.disk() as cache:
    await user_proxy.a_initiate_chat(  # noqa: F704
        coder,
        message="Create a timer for 5 seconds and then a stopwatch for 5 seconds.",
        cache=cache,
    )

user_proxy (to chatbot):

Create a timer for 5 seconds and then a stopwatch for 5 seconds.

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:06:16] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


chatbot (to user_proxy):

***** Suggested tool call (call_cwez): timer *****
Arguments: 
{"num_seconds":"5"}
**************************************************
***** Suggested tool call (call_68gd): stopwatch *****
Arguments: 
{"num_seconds":"5"}
******************************************************

--------------------------------------------------------------------------------
Replying as user_proxy. Provide feedback to chatbot. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit


**Router_tool:**

In [ ]:

from pydantic import BaseModel, Field
from typing_extensions import Annotated
# class Keyword(BaseModel):
  # keyword: Annotated[str, "keyword either 'vectorstore' or 'websearch'"]
def router_tool( query: Annotated[str, "question or query ask by user"])->str:
  """Router Function"""
  if 'ESOP' in query:
    return 'vectorstore'
  else:
    return 'websearch'


**Web Search tool  using tavily api :** Take your api from tavily.com

In [ ]:
#set your Api keys
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

search__tool=TavilySearchResults(k=3)

In [ ]:
search__tool.invoke("How does exercise price determine for ESOP?")

[{'url': 'https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf',
  'content': 'How does exercise price determine for ESOP? A. Companies are free to decide the exercise price, which may be issued at a discount or premium but the exercise price determined by the Company shall not be less than the par value of the shares. Q. Does it mandatory for the Company to issue and'},
 {'url': 'https://fi.money/guides/money-matters/how-to-determine-exercise-price-for-esop',
  'content': 'Ask Fi GuidesMoney MattersA Guide: How to Determine Exercise Price for ESOP? Make the Most of your Money with Fi Compare the stock’s fair market value to the exercise price as specified in the ESOP agreement. Make the Most of your Money with Fi Fi Money’s in-app AI assistant can help you manage your investments, build better monetary habits, meet all your financial goals & reward you for making noteworthy financial choices. Compare the exercise price in the agreement against the fair market value of your company

**creating web_search_tool**

In [ ]:
def docs2str(docs):
    return "\n\n".join(doc['content'] for doc in docs)
def web_search_tool(query: Annotated[str, "question or query ask by user"])->str:
  """Search Function"""
  docs=search__tool.run(query)
  return docs2str(docs)



In [ ]:
web_search_tool("who is current president of america?")

'National Union[l]\n1864\nAndrew Johnson\nDemocratic\n1872\nHenry Wilson[e]\nVacant\xa0afterNovember 22, 1875\nVacant\xa0afterNovember 25, 1885\n1900\nVacant\xa0afterNovember 21, 1899\nTheodore Roosevelt\n1904\nCharles W. Fairbanks\nVacant\xa0afterOctober 30, 1912\n1916\n1924\nCharles G. Dawes\n1936\n1940\n1944\nHenry A. Wallace\nHarry S. Truman\n1948\nAlben W. Barkley\n1956\n1964\nHubert Humphrey\n1972\nVacant:October 10\xa0– December 6, 1973\nGerald Ford[t]\nNelson Rockefeller[t]\n1984\n1996\n2004\n2012\nSee also\nNotes\nReferences\nWorks cited\nGeneral\nExpert studies\nPresidential biographies\nOnline sources\nExternal links He was, and remains, the only U.S. president never affiliated with a political party.[13]\nPresidents\n1792\n1804\nGeorge Clinton\n1812\nVacant\xa0afterApril 20, 1812\nElbridge Gerry[e]\nVacant\xa0afterNovember 23, 1814\n Contents\nList of presidents of the United States\nThe president of the United States is the head of state and head of government of the Unite

In [ ]:
web_search_tool("How does exercise price determine for ESOP?")

"How does exercise price determine for ESOP? A. Companies are free to decide the exercise price, which may be issued at a discount or premium but the exercise price determined by the Company shall not be less than the par value of the shares. Q. Does it mandatory for the Company to issue and\n\nAsk Fi GuidesMoney MattersA Guide: How to Determine Exercise Price for ESOP? Make the Most of your Money with Fi Compare the stock’s fair market value to the exercise price as specified in the ESOP agreement. Make the Most of your Money with Fi Fi Money’s in-app AI assistant can help you manage your investments, build better monetary habits, meet all your financial goals & reward you for making noteworthy financial choices. Compare the exercise price in the agreement against the fair market value of your company stock. Ask Fi Contact Fi Money customer care You can do everything from the Fi App, including p2p payments, fund transfers, bill payments, and more, with features to automate every actio

**Model name that we are using for agentic system**

In [ ]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent,ConversableAgent
# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

# config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['llama3-8b-8192']


**Agentic RAG Agents**

**The user sends a query to the UserProxyAgent**

In [ ]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="An intelligent proxy agent designed to bridge user requests and external AI systems seamlessly.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
)

**Router Agent:** The RouterAgent decides whether to route the query to the Vectorstore or WebSearch.

In [ ]:
from autogen import ConversableAgent

router_agent = ConversableAgent(
    name="Router Agent",
    system_message='''You are an expert at routing a user query to the appropriate source.
    Use tool router_tool for the routing descision only in one word  either vectorstore or websearch .
    Return 'TERMINATE' when routing is complete.''',
    #    Use the provided query inside router_tool for getting keyword vectorstore or webserach
    llm_config=llm_config,
    code_execution_config=False,
    # function_map={"router_tool": router_tool},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

**Resitering router_tool to Router Agent:**

In [ ]:


autogen.agentchat.register_function(
    router_tool,
    caller=router_agent,
    executor=user_proxy,
    description="Router Tool use to route the query",
)

In [ ]:
router_agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Router Tool use to route the query',
   'name': 'router_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}}]

In [ ]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    router_agent, message="who is the president of America", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Router Agent):

who is the president of America

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:37:32] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_521a): router_tool *****
Arguments: 
{"query":"Who is the president of America"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_521a) *****
websearch
**************************************************

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:37:33] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_bazx): router_tool *****
Arguments: 
{"query":"Who is the current President of the United States"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_bazx) *****
websearch
**************************************************

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:37:33] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_zdb3): router_tool *****
Arguments: 
{"query":"It is currently [insert current date] and the President of the United States is [insert current president name]"}
********************************************************
***** Suggested tool call (call_z113): router_tool *****
Arguments: 
{"query":"[insert current president name]"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_zdb3) *****
websearch
**************************************************

--------------------------------------------------------------------------------
***** Response from calling tool (call_z113) *****
websearch
**************************************************

-----------------------------------------

Router Agent (to user_proxy):

<tool-use>{"tool_calls":[]}</tool-use> TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:37:34] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


**Creating pdf_search_tool**

**Using langchain  for creating  vectorstore**

In [ ]:
import os
import shutil
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Initialize the embedding model
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def pdf_vectorstore(folder_path: str):
    if not os.path.isdir(folder_path):
        raise ValueError(f"Provided folder path '{folder_path}' does not exist or is not a directory.")

    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            print(f"Loading PDF file: {filename}")
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            print(f"Loading DOCX file: {filename}")
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Skipping unsupported file type: {filename}")
            continue
        documents.extend(loader.load())

    if not documents:
        raise ValueError("No valid documents found in the folder.")

    print("Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    print(f"Total chunks created: {len(splits)}")

    # Define persist directory
    persist_directory = "/tmp/chroma_db"

    # Ensure clean, writable directory
    if os.path.exists(persist_directory):
        print("Resetting persist directory...")
        shutil.rmtree(persist_directory)
    os.makedirs(persist_directory, exist_ok=True)

    if not os.access(persist_directory, os.W_OK):
        raise PermissionError(f"Directory '{persist_directory}' is not writable.")

    print("Initializing the vector store...")
    collection_name = "my_collection"
    vectorstore = Chroma.from_documents(
        collection_name=collection_name,
        documents=splits,
        embedding=embedding_function,
        persist_directory=persist_directory
    )
    print("Vector store successfully created.")
    return vectorstore


<ipython-input-15-88148e43040c>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print("https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf")

https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf


**Provide your folder path here**

In [ ]:
 # Load documents from a folder
folder_path = "/content/"  # Replace with the actual path to your folder
vectorstore=pdf_vectorstore(folder_path)

Skipping unsupported file type: .config
Skipping unsupported file type: .cache
Loading PDF file: 17.pdf
Skipping unsupported file type: sample_data
Splitting documents into chunks...
Total chunks created: 47
Initializing the vector store...
Vector store successfully created.


In [ ]:

from langchain.vectorstores.base import VectorStoreRetriever
retriever = VectorStoreRetriever(vectorstore=vectorstore)
# retriever.invoke(" How does exercise price determine for ESOP? ")
def docs2str_(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def pdf_search_tool(query: Annotated[str, "question or query ask by user"])->str:
  """pdf search"""
  docs=retriever.invoke(query)
  return docs2str_(docs)
pdf_search_tool(" How does exercise price determine for ESOP? ")

'of a higher salary, and the organization where they are \nemployed is not growing as per the market standards \nor in comparison to its competitors, ESOP may result \nin monetary loss.\nQ.  How does exercise price determine for ESOP?\nA. Companies are free to decide the exercise price, \nwhich may be issued at a discount or premium but the \nexercise price determined by the Company shall not \nbe less than the par value of the shares.\nQ.  Does it mandatory for the Company to issue and \nallot only fresh shares under ESOP scheme?\nA. No, it is not mandatory for the company to issue and \nallot only fresh shares under ESOP scheme, but it may \nchoose either option:\n 1.  If Company is willing to issue fresh shares, it \nshould adopt direct route to issue and allot shares \nunder the scheme of ESOP.\n 2. If the Company is willing to channelize its \nexisting share only, in such case Company should \nadopt Trust route to issue and allot shares under \nthe scheme of ESOP.\nQ.  How are ESO

**RetrieverAgent:** The RetrieverAgent processes the retrieved information and generates an answer using the Generator.

In [ ]:
retriever_agent = ConversableAgent(
    name="Retriever Agent",
    system_message='''You are responsible for retrieving relevant information from the given source.
    first modify the query for retrieving the information from vectorstore or websearch.
  keyword provide by Router Agent based on that if keyword is vectorstore then use pdf_search_tool(query) otherwise use web_search_tool(query) for retreiving the information related query
    Answer questions concisely using the retrieved context.
    Return 'TERMINATE' when retrieval is completed.''',
    llm_config=llm_config,
    code_execution_config=False,
    function_map={"web_search_tool": web_search_tool,"pdf_search_tool":pdf_search_tool},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),

)

**Resitering pdf_search_tool and  web_search_tool to Retriever Agent:**

In [ ]:
autogen.agentchat.register_function(
    pdf_search_tool,
    caller=retriever_agent,
    executor=user_proxy,
    description="Retrieve information  from vectorstore ",

)
autogen.agentchat.register_function(
    web_search_tool,
    caller=retriever_agent,
    executor=user_proxy,
    description="Retrieve information  from websearch ",

)
retriever_agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Retrieve information  from vectorstore ',
   'name': 'pdf_search_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}},
 {'type': 'function',
  'function': {'description': 'Retrieve information  from websearch ',
   'name': 'web_search_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}}]

In [ ]:

with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    retriever_agent, message="who is presisdent of america? provide clear and consie ", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Retriever Agent):

who is presisdent of america? provide clear and consie 

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:38:10] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Retriever Agent (to user_proxy):

***** Suggested tool call (call_4mec): web_search_tool *****
Arguments: 
{"query":"who is the president of the america"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_4mec) *****
The president is also the head of the executive branch of the federal government of the United States and is the chairman of the presidential cabinet.[10]
Joe Biden is the 46th and current president of the United States, in office since January 2021.[11]
Eligibility and requirements[change | change source]
Article II, Section 1, Clause 5 of the constitution states for a person to serve as president must:
Election process and presidential terms[change | change source]
The president is elected by the people through the Electoral College to a four-year term, along w

Retriever Agent (to user_proxy):

Joe Biden is the 46th and current President of the United States, in office since January 2021.

--------------------------------------------------------------------------------
user_proxy (to Retriever Agent):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:38:13] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Retriever Agent (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:38:14] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [ ]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    retriever_agent, message="""How does exercise price determine for ESOP?
    """, summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Retriever Agent):

How does exercise price determine for ESOP?
    

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:38:23] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Retriever Agent (to user_proxy):

***** Suggested tool call (call_5x50): pdf_search_tool *****
Arguments: 
{"query":"How does exercise price determine for ESOP?"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION pdf_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_5x50) *****
of a higher salary, and the organization where they are 
employed is not growing as per the market standards 
or in comparison to its competitors, ESOP may result 
in monetary loss.
Q.  How does exercise price determine for ESOP?
A. Companies are free to decide the exercise price, 
which may be issued at a discount or premium but the 
exercise price determined by the Company shall not 
be less than the par value of the shares.
Q.  Does it mandatory for the Company to issue and 
allot only fresh shares under ESOP scheme?
A. No, it is not mandatory for the 

Retriever Agent (to user_proxy):

Based on the output from the tool call, I'll provide the final answer.

According to the output, the exercise price determines the price payable by an employee for exercising the options granted in pursuance of the scheme of ESOP. Furthermore, the price is usually less than the share's Fair Market Value (FMV), and the difference between the FMV and the exercise price is considered a perquisite in the employee's hands and taxed at their income tax slab rate.

So, in simple terms, the exercise price is the price at which employees can buy the company's shares, and it's usually lower than the current market value.

--------------------------------------------------------------------------------
user_proxy (to Retriever Agent):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:38:24] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" 

Retriever Agent (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 01:38:25] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


**Answer Grader**: An answer grader agent that evaluates the relevance of the retrieved information

In [ ]:
answer_grader_agent = ConversableAgent(
    name="Answer Grader",
    system_message='''You assess whether the retrieved information from Retriever Agent is relevant to the query:(query).
    Use retrieved response from retriever_agent and use that information for further process
    If relevant, refine it for clarity; if not, suggest additional searches.
    Return 'TERMINATE' when grading is complete.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

**Hallucination Grader:** A hallucination grader agent that checks if the answer is grounded in the retrieved context

In [ ]:

hallucination_grader_agent = ConversableAgent(
    name="Hallucination Grader",
    system_message='''You ensure the answer is factually accurate and grounded in the retrieved context.
    Return 'TERMINATE' when verification is complete.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

**Final Answer Generator:** A final answer agent that generates a clear and concise final answer

In [ ]:
final_answer_agent = ConversableAgent(
    name="Final Answer Generator",
    system_message='''"You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool(query)' and based on that provide a response
        "Return 'TERMINATE' when verification is complete.''',
    llm_config=llm_config,
    code_execution_config=False,
    function_map={"web_search_tool": web_search_tool},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

**Resitering web_search_tool to Final Answer Generator**

In [ ]:

autogen.agentchat.register_function(
    web_search_tool,
    caller=final_answer_agent,
    executor=user_proxy,
    description="Retrieve information from websearch",

)
final_answer_agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Retrieve information from websearch',
   'name': 'web_search_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}}]

In [ ]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    final_answer_agent, message="who is presisdent of america?", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Final Answer Generator):

who is presisdent of america?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:08:17] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Final Answer Generator (to user_proxy):

***** Suggested tool call (call_7r10): web_search_tool *****
Arguments: 
{"query":"who is president of america"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...
user_proxy (to Final Answer Generator):

***** Response from calling tool (call_7r10) *****
Presidents | The White House The 4th President of the United States The 5th President of the United States The 6th President of the United States The 7th President of the United States The 8th President of the United States The 9th President of the United States The 10th President of the United States The 11th President of the United States The 12th President of the United States The 13th President of the United States The 14th President of the United States The 15th President of the United States The 16th President of the United States The 17th President o

**Creating Chat for Conversbale Agents**

In [ ]:
query="who wins election of 2024 of usa?"
chats = [
    # Step 1: User Proxy to Router Agent
    {
        "sender": user_proxy,
        "recipient": router_agent,
        "message":f"{query}",
        "summary_method": "last_msg",

        "max_turns": 2,
        "clear_history": True,
    },
    # Step 2: Router Agent to Retriever Agent
    {
        "sender": user_proxy,
        "recipient": retriever_agent,
        "message":f"""use the sender response as routing decision and query:{query}  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        """,
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
        # "carryover": "I want to include a use "
    },
    # Step 3: Retriever Agent to Answer Grader
    {
        "sender": user_proxy,
        "recipient": answer_grader_agent,
        "message": ("use the sender response as retrived content"
            f"Evaluate the relevance of the retrieved content from sender to the given query:{query}. "
            "Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. "
            "Do not provide explanations or additional information also do not provides python code "
        ),
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 4: Answer Grader Agent to Hallucination Grader Agent
    {
        "sender": user_proxy,
        "recipient": hallucination_grader_agent,
        "message": (
            "Evaluate whether the response provided by sender is grounded in the retrieved context. "
            "Provide a binary score: 'yes' or 'no'. "
            "Do not include preamble or explanations."
        ),
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 5: Hallucination Grader Agent to Final Answer Agent
    {
        "sender": user_proxy,
        "recipient": final_answer_agent,
        "message": (
            "If the response from sender is 'yes', generate a clear and concise final answer. "
            "If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search."
        ),
        "summary_method": "last_msg",
        "max_turns": 3,
        "clear_history": False,
    },
    # Step 6: Final Answer Agent to User Proxy
    {
        "sender": final_answer_agent,
        "recipient": user_proxy,
        "message": (
            "Here is the finalized answer to your query. I hope this resolves your question effectively."
        ),
        "summary_method": "last_msg",
        "max_turns": 1,
        "clear_history": False,
    },
]


**Initiate the chats:** for web base question

In [ ]:

from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Router Agent):

who wins election of 2024 of usa?

--------------------------------------------------------------------------------
Router Agent (to user_proxy):

***** Suggested tool call (call_vmdk): router_tool *****
Arguments: 
{"query":"who wins election of 2024 of usa? vectorstore"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_vmdk) *****
websearch
**************************************************

--------------------------------------------------------------------------------
Router Agent (to user_proxy):

***** Suggested tool call (call_09te): router_tool *****
Arguments: 

Retriever Agent (to user_proxy):

***** Suggested tool call (call_nr2a): web_search_tool *****
Arguments: 
{"query":"who wins election of 2024 of usa"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_nr2a) *****
Trump wins 2024 U.S. presidential election - Los Angeles Times Trump elected 47th president of the United States, defeating Harris to retake White House Nancy Northup, President and CEO of the Center for Reproductive Rights, called Trump’s win “a deadly threat to the democratic values of liberty and equality, the rule of law, and reproductive health, rights, and justice in the United States and around the globe.” “The Democrats thought there were enough people who hated Trump or were willing to fear him to win the race,” Jennings said. 2024 election results: Trump w

Retriever Agent (to user_proxy):

<tool-use>{"tool_calls":[],"output":"Donald Trump has won the 2024 Presidential Election."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Answer Grader):

use the sender response as retrived contentEvaluate the relevance of the retrieved content from sender to the given query:who wins election of 2024 of usa?. Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. Do not provide explanations or additional information also do not provides python code 
Context: 

<tool-use>{"tool_calls":[],"output":"Donald Trump has won the 2024 Presidential Election."}</tool-use>

Note: 

Answer Grader (to user_proxy):

yes

--------------------------------------------------------------------------------
user_proxy (to Answer Grader):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:13:59] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Answer Grader (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Hallucination Grader):

Evaluate whether the response provided by sender is grounded in the retrieved context. Provide a binary score: 'yes' or 'no'. Do not include preamble or explanations.
Context: 

<tool-use>{"tool_calls":[],"output":"Donald Trump has won the 2024 Presidential Election."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.


--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:14:00] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, compl

Hallucination Grader (to user_proxy):

no

--------------------------------------------------------------------------------
user_proxy (to Hallucination Grader):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:14:00] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Hallucination Grader (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Final Answer Generator):

If the response from sender is 'yes', generate a clear and concise final answer. If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search.
Context: 

<tool-use>{"tool_calls":[],"output":"Donald Trump has won the 2024 Presidential Election."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:14:00] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"p

Final Answer Generator (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Final Answer Generator (to user_proxy):

Here is the finalized answer to your query. I hope this resolves your question effectively.
Context: 

<tool-use>{"tool_calls":[],"output":"Donald Trump has won the 2024 Presidential Election."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.




--------------------------------------------------------------------------------
user_proxy (to Final Answer Generator):



--------------------------------------------------------------------------------


**Initiate the chats:** for knowledge base question

In [ ]:
query="How does exercise price determine for ESOP?"
chats = [
    # Step 1: User Proxy to Router Agent
    {
        "sender": user_proxy,
        "recipient": router_agent,
        "message":f"{query}",
        "summary_method": "last_msg",

        "max_turns": 2,
        "clear_history": True,
    },
    # Step 2: Router Agent to Retriever Agent
    {
        "sender": user_proxy,
        "recipient": retriever_agent,
        "message":f"""use the sender response as routing decision and query:{query}  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        """,
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
        # "carryover": "I want to include a use "
    },
    # Step 3: Retriever Agent to Answer Grader
    {
        "sender": user_proxy,
        "recipient": answer_grader_agent,
        "message": ("use the sender response as retrived content"
            f"Evaluate the relevance of the retrieved content from sender to the given query:{query}. "
            "Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. "
            "Do not provide explanations or additional information also do not provides python code "
        ),
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 4: Answer Grader Agent to Hallucination Grader Agent
    {
        "sender": user_proxy,
        "recipient": hallucination_grader_agent,
        "message": (
            "Evaluate whether the response provided by sender is grounded in the retrieved context. "
            "Provide a binary score: 'yes' or 'no'. "
            "Do not include preamble or explanations."
        ),
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 5: Hallucination Grader Agent to Final Answer Agent
    {
        "sender": user_proxy,
        "recipient": final_answer_agent,
        "message": (
            "If the response from sender is 'yes', generate a clear and concise final answer. "
            "If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search."
        ),
        "summary_method": "last_msg",
        "max_turns": 3,
        "clear_history": False,
    },
    # Step 6: Final Answer Agent to User Proxy
    {
        "sender": final_answer_agent,
        "recipient": user_proxy,
        "message": (
            "Here is the finalized answer to your query. I hope this resolves your question effectively."
        ),
        "summary_method": "last_msg",
        "max_turns": 1,
        "clear_history": False,
    },
]


In [ ]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Router Agent):

How does exercise price determine for ESOP?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:12:49] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_3mpb): router_tool *****
Arguments: 
{"query":"How does exercise price determine for ESOP?"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_3mpb) *****
vectorstore
**************************************************

--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:12:49] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_jspq): router_tool *****
Arguments: 
{"query":"What is exercise price in ESOP context?"}
********************************************************

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Retriever Agent):

use the sender response as routing decision and query:How does exercise price determine for ESOP?  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        
Context: 


--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:12:51] {410

Retriever Agent (to user_proxy):

***** Suggested tool call (call_pbvy): pdf_search_tool *****
Arguments: 
{"query":"How does exercise price determine for ESOP?"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION pdf_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_pbvy) *****
of a higher salary, and the organization where they are 
employed is not growing as per the market standards 
or in comparison to its competitors, ESOP may result 
in monetary loss.
Q.  How does exercise price determine for ESOP?
A. Companies are free to decide the exercise price, 
which may be issued at a discount or premium but the 
exercise price determined by the Company shall not 
be less than the par value of the shares.
Q.  Does it mandatory for the Company to issue and 
allot only fresh shares under ESOP scheme?
A. No, it is not mandatory for the 

Retriever Agent (to user_proxy):

<tool-use>{"tool_calls":[],"output":"The exercise price is determined by the company and shall not be less than the par value of the shares. It is also not mandatory for the company to issue and allot only fresh shares under ESOP scheme."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Answer Grader):

use the sender response as retrived contentEvaluate the relevance of the retrieved content from sender to the given query:How does exercise price determine for ESOP?. Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. Do not provide explanations or additional informati

...

Answer Grader (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Hallucination Grader):

Evaluate whether the response provided by sender is grounded in the retrieved context. Provide a binary score: 'yes' or 'no'. Do not include preamble or explanations.
Context: 

<tool-use>{"tool_calls":[],"output":"The exercise price is determined by the company and shall not be less than the par value of the shares. It is also not mandatory for the company to issue and allot only fresh shares under ESOP scheme."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.


--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13

Hallucination Grader (to user_proxy):

yes

--------------------------------------------------------------------------------
user_proxy (to Hallucination Grader):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:12:53] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Hallucination Grader (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Final Answer Generator):

If the response from sender is 'yes', generate a clear and concise final answer. If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search.
Context: 

<tool-use>{"tool_calls":[],"output":"The exercise price is determined by the company and shall not be less than the par value of the shares. It is also not mandatory for the company to issue and allot only fresh shares under ESOP scheme."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.



---------------------------------------------------------------------------

Final Answer Generator (to user_proxy):

The current President of the United States is Joe Biden.

--------------------------------------------------------------------------------
user_proxy (to Final Answer Generator):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-03 13:12:55] {410} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Final Answer Generator (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Final Answer Generator (to user_proxy):

Here is the finalized answer to your query. I hope this resolves your question effectively.
Context: 

<tool-use>{"tool_calls":[],"output":"The exercise price is determined by the company and shall not be less than the par value of the shares. It is also not mandatory for the company to issue and allot only fresh shares under ESOP scheme."}</tool-use>

Note: The conversation has ended. The final output is a concise answer based on the retrieved information.




--------------------------------------------------------------------------------
user_proxy (to Final Answer Generator):



------------------------------------------------